# Loading datas from .ods

In [1]:
# https://pypi.org/project/pandas-ods-reader/
# !pip install pandas_ods_reader
from pandas_ods_reader import read_ods

import pandas as pd
import numpy as np

# display non-truncated dataframe
pd.set_option('display.max_colwidth', -1)

/Users/francois/anaconda3/envs/data-science/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if __name__ == '__main__':


In [2]:
path = "data/88milSMS_88522.ods"

# load a sheet based on its index (1 based)
sheet_idx = 1
df = read_ods(path, sheet_idx)

# load a sheet based on its name
sheet_name = "Feuille1"
df = read_ods(path, sheet_name)

# load a file that does not contain a header row
# if no columns are provided, they will be numbered
df = read_ods(path, 1, headers=True)

# load a file and provide custom column names
# if headers is True (the default), the header row will be overwritten
# df = read_ods(path, 1, columns=["NUM_SMS", "DATETIME", "ID_NUM_TEL", "SMS_ANON"])

In [3]:
df.head()

,NUM_SMS,DATETIME,ID_NUM_TEL,SMS_ANON
0,1.0,15 sept. 2011 07:28:55,477.0,"Hey ca va?\nAlors cette rentree?\nVa falloir se trouver un p'tit creneau pour se voir!\nDis moi, est-ce que tu sais quand commence les cours de langue non specialiste?\nBisoux"
1,2.0,15 sept. 2011 08:02:08,477.0,Ok super merci! Oui j'y comprends rien du tout moi a cette fac :p\n\nJ'irai me renseigner aussi ( si j'trouve le batiment :) )\nMerci encore!\n\nBonne soiree bisoux!
2,3.0,15 sept. 2011 08:03:01,477.0,"Coucou !\nC'est quand la feria de Nimes? \nJ'suis pas bien sure de la faire mais pourquoi pas!\nSinon oui, j'profite de ma derniere semaine de vacance!\nBon courage!\nBisoux"
3,4.0,15 sept. 2011 08:03:35,477.0,Coucou :)\nOui ca c'est bien passe!\nAlors je sais pas du tout pour jeudi soir... Non pas que je veuille pas mais je suis pas sure d'etre sur Montpel. J'y serai de sur qu'a partir de lundi prochain car je commence les cours...
4,5.0,15 sept. 2011 09:05:28,477.0,On peut se rejoindre quelque part? Tu as cours ou ?


## Convert types and format date

In [4]:
df["NUM_SMS"] = df["NUM_SMS"].astype('int64')
df["ID_NUM_TEL"] = df["ID_NUM_TEL"].astype('int64')

df['day'] = (df["DATETIME"].str.split(" ", n = 3, expand = True))[0]
df['month'] = (df["DATETIME"].str.split(" ", n = 3, expand = True))[1]
df['year'] = (df["DATETIME"].str.split(" ", n = 3, expand = True))[2]
df['time'] = (df["DATETIME"].str.split(" ", n = 3, expand = True))[3]

df["day"] = df["day"].astype('int64')
df["year"] = df["year"].astype('int64')

def month_to_num(item):
    MONTH_NUMS = {
        'jan.': 1,
        'fév.': 2,
        'mar.': 3,
        'avr.': 4,
        'mai.': 5,
        'juin.': 6,
        'juil.': 7,
        'aout.': 8,
        'sept.': 9,
        'oct.': 10,
        'nov.': 11,
        'déc.': 12,
    }
    return MONTH_NUMS.get(item)

df['month'] = df.apply(lambda x: month_to_num(x['month']),axis=1)

In [5]:
df.dtypes

NUM_SMS       int64 
DATETIME      object
ID_NUM_TEL    int64 
SMS_ANON      object
day           int64 
month         int64 
year          int64 
time          object
dtype: object

In [6]:
# Split date
df['hour'] = (df["time"].str.split(":", n = 2, expand = True))[0]
df['minute'] = (df["time"].str.split(":", n = 2, expand = True))[1]
df['second'] = (df["time"].str.split(":", n = 2, expand = True))[2]

# Create new date
df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute', 'second']])

In [7]:
df.dtypes

NUM_SMS       int64         
DATETIME      object        
ID_NUM_TEL    int64         
SMS_ANON      object        
day           int64         
month         int64         
year          int64         
time          object        
hour          object        
minute        object        
second        object        
date          datetime64[ns]
dtype: object

## Création timestamp

In [8]:
df['ts'] = df.date.values.astype(np.int64)

In [9]:
df.dtypes

NUM_SMS       int64         
DATETIME      object        
ID_NUM_TEL    int64         
SMS_ANON      object        
day           int64         
month         int64         
year          int64         
time          object        
hour          object        
minute        object        
second        object        
date          datetime64[ns]
ts            int64         
dtype: object

In [10]:
toyset = df[['NUM_SMS','SMS_ANON','ID_NUM_TEL']].head(5)
toyset

,NUM_SMS,SMS_ANON,ID_NUM_TEL
0,1,"Hey ca va?\nAlors cette rentree?\nVa falloir se trouver un p'tit creneau pour se voir!\nDis moi, est-ce que tu sais quand commence les cours de langue non specialiste?\nBisoux",477
1,2,Ok super merci! Oui j'y comprends rien du tout moi a cette fac :p\n\nJ'irai me renseigner aussi ( si j'trouve le batiment :) )\nMerci encore!\n\nBonne soiree bisoux!,477
2,3,"Coucou !\nC'est quand la feria de Nimes? \nJ'suis pas bien sure de la faire mais pourquoi pas!\nSinon oui, j'profite de ma derniere semaine de vacance!\nBon courage!\nBisoux",477
3,4,Coucou :)\nOui ca c'est bien passe!\nAlors je sais pas du tout pour jeudi soir... Non pas que je veuille pas mais je suis pas sure d'etre sur Montpel. J'y serai de sur qu'a partir de lundi prochain car je commence les cours...,477
4,5,On peut se rejoindre quelque part? Tu as cours ou ?,477


## Connect to database

In [11]:
# todo
# https://pynative.com/python-mysql-database-connection/
# !pip install mysql-connector-python

import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(host='localhost',
                                         database='master_p',
                                         user='root',
                                         password='xxxx')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")


Error while connecting to MySQL 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)


NameError: name 'connection' is not defined

## Insert into database

In [12]:
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='master_p',
                                         user='root',
                                         password='xxxx')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        
        sql = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'master_p' AND TABLE_NAME = 'SMS';"
        cursor.execute(sql)
        # Fetch all the records
        result = cursor.fetchall()
#         print(result)
        for i in result:
            print(i)
        
except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.21
You're connected to database:  ('master_p',)
('id_sms',)
('datetime',)
('message',)
('UTILISATEUR_id_utilisateur',)
('SUJET_id_sujet',)
('ACTIVITE_id_activite',)
MySQL connection is closed


## Insert

In [13]:
# db_cols = "id_sms,datetime,message,UTILISATEUR_id_utilisateur,SUJET_id_sujet,ACTIVITE_id_activite,ACTIVITE_SUJET_id_sujet"

# print(toyset[:1])

try:
    connection = mysql.connector.connect(host='localhost',
                                         database='master_p',
                                         user='root',
                                         password='xxxx')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        
        # Insert DataFrame recrds one by one.
        for i,row in toyset.iterrows():
            sql = "INSERT INTO SMS(id_sms,message,UTILISATEUR_id_utilisateur) VALUES (%s,%s,%s)"
            cursor.execute(sql, tuple(row))

            # the connection is not autocommitted by default, so we must commit to save our changes
            connection.commit()
                
except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")


Connected to MySQL Server version  8.0.21
You're connected to database:  ('master_p',)
MySQL connection is closed


In [19]:
# todo

try:
    connection = mysql.connector.connect(host='localhost',
                                         database='master_p',
                                         user='root',
                                         password='xxxx')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        
        # Select 
        sql = "SELECT * FROM SMS;"
        cursor.execute(sql)
        
        print("\n")
        myresult = cursor.fetchall()
        for x in myresult:
          print(x, "\n")

except Error as e:
    print("Error while connecting to MySQL", e)
    
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.21
You're connected to database:  ('master_p',)


(1, None, b"Hey ca va?\nAlors cette rentree?\nVa falloir se trouver un p'tit creneau pour se voir!\nDis moi, est-ce que tu sais quand commence les cours de langue non specialiste?\nBisoux", 477, None, None) 

(2, None, b"Ok super merci! Oui j'y comprends rien du tout moi a cette fac :p\n\nJ'irai me renseigner aussi ( si j'trouve le batiment :) )\nMerci encore!\n\nBonne soiree bisoux!", 477, None, None) 

(3, None, b"Coucou !\nC'est quand la feria de Nimes? \nJ'suis pas bien sure de la faire mais pourquoi pas!\nSinon oui, j'profite de ma derniere semaine de vacance!\nBon courage!\nBisoux", 477, None, None) 

(4, None, b"Coucou :)\nOui ca c'est bien passe!\nAlors je sais pas du tout pour jeudi soir... Non pas que je veuille pas mais je suis pas sure d'etre sur Montpel. J'y serai de sur qu'a partir de lundi prochain car je commence les cours...", 477, None, None) 

(5, None, b'On peut se rejoindre que